### Initialize

In [1]:
import sys
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons, make_circles, make_classification 
from sklearn import metrics
from sklearn import linear_model
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from dask.distributed import Pub, Sub, TimeoutError
from dask.distributed import get_worker,wait
import dask
import dask.array as da
import statistics
import math 
import asyncio
import time
import random
import re

### Dask client

In [2]:
from dask.distributed import Client , LocalCluster

cluster = LocalCluster(n_workers=5, threads_per_worker=1)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:43472 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 33.69 GB


### Αssign each worker to a local name

In [3]:
c=cluster.scheduler.workers
worker1= c.items()[0][0]
worker2= c.items()[1][0]
worker3= c.items()[2][0]
worker4= c.items()[3][0]
worker5= c.items()[4][0]

## Pub-Sub structure
***
### Coordinator
Coordinator is used by a future, in specific worker (worker1).First send E if E=0,askes workers to compute local drifts in order to update E.This future ends every time that we get a new E. Starts again till workers have no chunks to read 

In [4]:
def coordinator(n_workers,E):
    pub_init = Pub('Initialize')
    pub_th = Pub('Theta')
    pub_endr = Pub('EndRound')
    pub_endsub = Pub('EndSubRound')
    sub_incr = Sub('Increment')
    sub_f = Sub('Fs')
    sub_x = Sub('Xs')
    #--------------------------------Useful functions------------------------------------
    def get_incr():    
        try:
            incr=sub_incr.get(timeout=15)
            return incr
        except TimeoutError:
            print('Increment aknowlegment not received')
            return -10
    def get_fi():  
        fis=[]
        try:
            fi=sub_f.get()
            fis.append(fi)
            for i in range(3):
                fi=sub_f.get() 
                fis.append(fi)
            return fis
        except:
            print('Fi aknowlegment not received')
            return -10
    def get_xi():  
        drifts=[]
        try:
            xi=sub_x.get()
            drifts.append(xi)
            for i in range(3):
                xi=sub_x.get() 
                drifts.append(xi)
            return drifts
        except:
            print('Xi aknowlegment not received')
            return -10
    def longest(l):
        max_len=0
        for i in l:
            if len(i)>max_len:
                max_len=len(i)
        return max_len
    
    def add_x(array):
        
        length=longest(array)
        sum_x=[0]*length
        for x in array:
            dif=length-len(x)
            for i in range(dif):
                x=np.append(x,0)
            sum_x=np.add(sum_x,x)
        return sum_x
    
    def add_f(array):
        sum_x=0
        for x in array:
            sum_x=sum_x+x
        return sum_x
    #------------------------------Start coordinator--------------------------------------
    while len(pub_init.subscribers)!=4:
        #if not all workers subscribe sleep
        time.sleep(0.01)
    
    counter=0
    count_sub=0
    k=n_workers
    th=0
#     c=0
    fis=0
    drifts=0
    sum_xi=0
    incr=0
    e_y=0.01
    print("Coo started")
    if E is 0: #if E=0 we need to update E
        pub_init.put(None) 
        drifts=get_xi() #get local drifts (Xi's)
        sum_xi=add_x(drifts)
        E=E+sum_xi
        pub_init.put(E)
    else:
        pub_init.put(E)
    
    print("E",E)
    y=k*f([0],E)
    subs=[]
    flag=True #use this flag to finish future if chunks are out
    while y<=e_y*k*f([0],E): #start of the round
        
        counter=counter+1
        print("START ROUND:",counter)
        th=-y/(2*k)
        print("th",th)
        
        pub_endr.put(1) #let worker know that a new round begins
        pub_th.put(th) #send theta
        
        c=0
        fis=[]
        count_sub=0
        
        while c<k: #start of the subround
            
            count_sub=count_sub+1
            print("START SUBROUND:",count_sub)
            
            pub_endsub.put(1) #let worker know that a new subround begins
            incr=get_incr() #Get increments
#             print("incr")
            if type(incr)==str: # works as a flag to let coordinator know that chunks are out
                print("Coo received notice of chunks ended...")
#                 pub_endsub.put(0)
                flag=False 
                break
            if incr==-10:
                incr=0
            
            c=c+incr
            print("c,incr",c,incr)
#-----------------------------subrounds ended---------------------------------------------
        if flag==False: #if false chunks are out end future
#             pub_endsub.put(0)
            pub_endsub.put(0)
            break
    
        pub_endsub.put(0) #let workers know that subrounds ended 
        fis=get_fi() #get F(Xi)'s from workers
        
        y=add_f(fis)
        print("y",y)
#-----------------------------rounds ended-------------------------------------------------
    pub_endr.put(0) #let workers know that rounds ended 
    
    drifts=get_xi() #get local drifts (Xi's)
    sum_xi=add_x(drifts)
    E=E+sum_xi
        
    print("Coo ended...")
    return E,counter,count_sub
    

### Worker
***
Worker function is used in multiple future (here 4), each one is assigned in a different cluster worker.Workers run till there are now more chunks.There are now results needed from this futures so, there is no need to restart them (like we do with the coordinator). Each worker reads a different file to update Si. 

In [5]:
def worker_f(name,Si,clf):
    sub_init = Sub('Initialize')
    sub_th = Sub('Theta')
    sub_endr = Sub('EndRound')
    sub_endsub = Sub('EndSubRound')
    pub_incr = Pub('Increment')
    pub_f = Pub('Fs')
    pub_x = Pub('Xs')
    #--------------------------------Useful functions------------------------------------
    def get_init():    
        try:
            init=sub_init.get(timeout=100)
            return init
        except TimeoutError:
            print('Init aknowlegment not received')
            return []
        
    def get_th():    
        try:
            th=sub_th.get(timeout=100)
            return th
        except TimeoutError:
            print('Theta aknowlegment not received')
            return -10
        
    def get_endr():
        try:
            endr=sub_endr.get(timeout=100)
            return endr
        except TimeoutError:
            print('EndofRound aknowlegment not received')
            return -10
    
    def get_endsub():
        try:
            endsub=sub_endsub.get(timeout=100)
            return endsub
        except TimeoutError:
            print('EndofSubRound aknowlegment not received')
            return -10
        
    def make_same(x1,x2):
        if len(x1)!=len(x2):
            dif=len(x2)-len(x1)
            
            for i in range(abs(dif)):
                if dif>0:
                    x1=np.append(x1,10)
                else:

                    x2=np.append(x2,0)
        return x1,x2
    #--------------------------------Start of worker------------------------------------
    start=False
    th=0
    w_id= get_worker().name #get worker id
    f_name="sample"+str(w_id)+".txt" # concat sample with the w_id to get file name
    flag=True 
    E=[]
    Xi=[]
    
    count_chunks=1
    while flag==True: #while this flag stays true there are chunks
        
        E=get_init() # get E from coordinator

        if E is None: #if E=0 compute Xi and return Xi to update E
            X,y=get_chunk(count_chunks) #get_newSi(count_chunks,f_name)
            if type(X)==str and type(y)==str:
                flag=False
                print("NO Chunks,hey")
                break
            clf.partial_fit(X,y,np.unique(([0,1])))
            Si = clf.coef_[0]
            Xi=Si
            pub_x.put(Xi)
            E=get_init()

#----------------------------------begin of round----------------------------------------
        while get_endr()==1: 
            ci=0
            Xi=[0]

            th=get_th() #get theta

            if th==-10:
                break
#----------------------------------begin of subround----------------------------------------
            while get_endsub()==1:

                count_chunks=count_chunks+1

                zi=f(Xi,E)
                X,y=get_chunk(count_chunks) #get_newSi(count_chunks,f_name)
                if type(X)==str and type(y)==str:
                    flag=False
                    print("NO Chunks")
                    pub_incr.put("no")
                else:
                
                    clf.partial_fit(X,y,np.unique([0,1]))
                    new=clf.coef_[0]

                    Si,new=make_same(Si,new) #read from the file
                    S_prev=Si 
                    Si=np.add(Si,new)
                    Xi=Si-S_prev
                    print(w_id,"Xi",Xi)
                    c_th=0 

                    if th!=0: #avoid division with 0 if th=0 c_th=0
                        c_th=(f(Xi,E)-zi)/th
                    print(w_id,"c_th",c_th)
                    ci_new=max(ci,math.floor(c_th))
                    print(w_id,"ci",ci,"new ci",ci_new)
                    if ci!=ci_new: #if we detect a difference send it to the coordinator
                        ci=ci_new
                        pub_incr.put(ci)
    #                 else:
    #                     pub_incr.put(0)
                        print(w_id,"Sended...",ci)
#----------------------------------end of subround----------------------------------------
#             if flag==False: #if flag is false let coordinator know
#                 pub_incr.put("no")
#                 print("No chunks, end of round")
#                 time.sleep(3)
#                 break
            
            pub_f.put(f(Xi,E)) 
#-------------------------------end of round----------------------------------------
        time.sleep(4)
        pub_x.put(Xi) # send Xi    
    #time.sleep(3)
    print(w_id,"Ended...")
    return Si
        

## Functions
***
### Φ(x)

In [6]:
def f(x,E):
    if len(x)!=len(E):
        dif=len(E)-len(x)
        for i in range(abs(dif)):
            if dif>0:
                x=np.append(x,0)
            else:
                E=np.append(E,0)
    if LA.norm(E)!=0:
        
        t1= -0.01*LA.norm(E)-np.dot(x,(E/LA.norm(E)))
    else:
        t1=0
    n_sum=np.add(x,E)
    t2=LA.norm(n_sum) - (1+0.01)*LA.norm(E)
    return max(t1,t2)
print(f([0.1,0,0.1,0.1,0,0,0.1,0,0,0,0,0,0,0],[0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

-0.035003121099842716


### Get new Si 
***
Read from file a spesific line and convert the string to np.array

file form: [0,0,0,0,0,0,0,0,0,0,0] [0,0,0,0,0,0,0,0,0,0,0.1]... 20 line each

In [7]:
def get_newSi(x,f_name):
    fp = open(f_name)
    l=None    
    for i, line in enumerate(fp):
        if line in ['\n', '\r\n']: #exclude [ ,]
            break
        if i == x-1:
            l=re.sub('\]|\[','', line)
            l= list(l.split(",")) #split string
            for i in range(len(l)): #convert from char to float
                l[i] = float(l[i])
            break      
    fp.close()
    return l

### Get Chunk 
***

In [8]:
def get_chunk(n):
    X=np.load("X.npy")
    y=np.load("y.npy")
    n=n-1
    n_parts=200
    split= int(len(X)/n_parts)
    start=split*n
    end=start+split
    if start>=(len(X)-1):
        return "False","False"
    if end>(len(X)-1):
        end=len(X)
    X_train = X[start:end]
    y_train = y[start:end]
    return X_train,y_train

### Future start function 
***
Starts 4 workers and one coordinator worker 

In [9]:
def future_start(E,Si,clf):
    coo= client.submit(coordinator,4,E,workers=worker1)
    w1=client.submit(worker_f,1,Si[0],clf,workers=worker2)
    w2=client.submit(worker_f,2,Si[1],clf, workers=worker3)
    w3=client.submit(worker_f,3,Si[2],clf, workers=worker4)
    w4=client.submit(worker_f,4,Si[3],clf, workers=worker5)
    return coo,w1,w2,w3,w4

In [10]:
import traceback
def main():
    #user_input=input("Enter")
    #make a dataset and save training X and y 
    X, y = make_classification(n_samples=1200, n_features=4,n_classes=2)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    np.save("X", X_train)
    np.save("y", y_train)
    np.save("X_test", X_test)
    np.save("y_test", y_test)
#     print("X y: ",X_train,y_train)
    clf = linear_model.SGDClassifier()
    coo,w1,w2,w3,w4=future_start(0,[[0,0],[0,0],[0,0],[0,0]],clf)
    print("coo",coo.result())
    future=[coo,w1,w2,w3,w4]
    while True:
        status_l=[w1.status,w2.status,w3.status,w4.status]
#         print("status: ",status_l)
        if status_l.count('error')==len(status_l):
            print("ERROR:\n",w1.traceback(),"\n with format:\n",traceback.format_tb(w1.traceback()) )
        if status_l.count('finished')!=len(status_l):
            if coo.status=='finished':
                print("coo",coo.result())
                E=coo.result()[0]
                del coo
                coo= client.submit(coordinator,4,E,workers=worker1)

        else:
            print("End of chunks.For now on E will stay the same!")
            status_l=[coo.status,w1.status,w2.status,w3.status,w4.status]
            print(status_l)
            while coo.status!='finished':
                time.sleep(0.01)
            status_l=[coo.status,w1.status,w2.status,w3.status,w4.status]
            print(status_l)
            print("coo",coo.result())
            break
        # here we will predict
#         user_input = input("Enter: ") 
    
    future=[coo,w1,w2,w3,w4]
    for f in future: del f

In [11]:
main()

coo (array([-137.40543803, -161.2029959 ,  137.37089331,   52.03545197]), 1, 1)
coo (array([-137.40543803, -161.2029959 ,  137.37089331,   52.03545197]), 1, 1)
coo (array([-110.3553591 , -241.3834686 ,  247.12209504,   67.59397969]), 1, 13)
coo (array([ -60.23800098, -308.88879081,  351.39732924,   77.73393432]), 1, 1)
coo (array([ -13.05362586, -384.82110553,  464.69925714,   90.1341545 ]), 1, 1)
coo (array([  81.39877622, -420.71108929,  549.5833521 ,   88.18728099]), 1, 1)
coo (array([ 183.52516684, -461.47196354,  643.75355544,   86.5329442 ]), 1, 1)
coo (array([ 249.66748211, -541.90196823,  770.78545115,   97.91853753]), 1, 1)
coo (array([ 345.91797522, -591.39737603,  873.08030422,   98.91384557]), 1, 1)
coo (array([ 441.81428068, -640.710648  ,  974.9987273 ,   99.90549102]), 1, 1)
coo (array([ 537.35899566, -689.84311972, 1076.54348062,  100.89350074]), 1, 1)
coo (array([ 632.55468885, -738.79611208, 1177.71729421,  101.87790129]), 1, 1)
coo (array([ 727.4039009 , -787.5709315

### Partial fit with the same data-compaire results
***

In [12]:
# X,y=get_chunk(2) #get_newSi(count_chunks,f_name)
# if type(X)==str and type(y)==str:
#     flag=False
#     print("NO Chunks")
# clf.partial_fit(X,y,np.unique([0,1]))
# return clf,clf.coef_,X_test,y_test

In [13]:
# from sys import getsizeof
# def kati():
#     clf = linear_model.SGDClassifier()
#     X, y = make_classification(n_samples=1200, n_features=4,n_classes=2)
#     X_train, X_test, y_train, y_test = train_test_split(X, y)
#     np.save("X", X_train)
#     np.save("y", y_train)
#     X,y=get_chunk(2) #get_newSi(count_chunks,f_name)
#     if type(X)==str and type(y)==str:
#         flag=False
#         print("NO Chunks")

#     clf.partial_fit(X,y,np.unique([0,1]))
#     return clf,clf.coef_,X_test,y_test
# # getsizeof(clf),getsizeof(clf.coef_)

In [14]:
# clf1,coef,X_test,y_test=kati()
# getsizeof(clf2),getsizeof(coef),clf2.coef_ , coef
import sys
from types import ModuleType, FunctionType
from gc import get_referents

# Custom objects know their class.
# Function objects seem to know way too much, including modules.
# Exclude modules as well.
BLACKLIST = type, ModuleType, FunctionType


def getsize(obj):
    """sum size of object & members."""
    if isinstance(obj, BLACKLIST):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            
            if not isinstance(obj, BLACKLIST) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
        objects = get_referents(*need_referents)
    return size

#_______________________________Compaire size_____________________________________
# all_s=getsize(coef)+getsize(clf2.intercept_)
# intercept=clf1.intercept_[0].item()
# coef_list=coef[0].tolist()
# coef[0],coef_list,getsizeof(coef[0]),getsizeof(coef_list),getsizeof(clf1.intercept_[0]),getsizeof(intercept)


# pyval = val.item()
# print(type(pyval))
#________________________________Compaire predict________________________________________
# clf2=linear_model.SGDClassifier()
# clf2=clf
# clf2.coef_=clf1.coef_
# clf2.intercept_=clf1.intercept_
# # clf2.t_=clf1.t_

# y_pred1 = clf1.predict(X_test)
# y_pred2 = clf2.predict(X_test)
# sys.stdout.write("\nAccuracy: %f" % (100*metrics.accuracy_score(y_test, y_pred1)))
# sys.stdout.write("\nAccuracy: %f" % (100*metrics.accuracy_score(y_test, y_pred2)))